**1-explode question**


In [0]:
data=[('Alice','badminton,tennis'),('Bob','swimming,football'),('Julie','cricket,boxing')]
columns=['name','hobbies']
df_hobbies=spark.createDataFrame(data,columns)
df_hobbies.show(truncate=False)

+-----+-----------------+
|name |hobbies          |
+-----+-----------------+
|Alice|badminton,tennis |
|Bob  |swimming,football|
|Julie|cricket,boxing   |
+-----+-----------------+



In [0]:
from pyspark.sql.functions import *

df_hobb1=df_hobbies.withColumn('hobbies',split(col('hobbies'),","))
display(df_hobb1)
df_hobb1.printSchema()

name,hobbies
Alice,"List(badminton, tennis)"
Bob,"List(swimming, football)"
Julie,"List(cricket, boxing)"


root
 |-- name: string (nullable = true)
 |-- hobbies: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [0]:

display(df_hobb1.withColumn('hobbies',explode('hobbies')))

name,hobbies
Alice,badminton
Alice,tennis
Bob,swimming
Bob,football
Julie,cricket
Julie,boxing


**2-coalesce question**

In [0]:
data2=[('Goa', ' ', 'AP'),('', 'AP', None), (None, ' ', 'Bglr')]
columns2=["city1", "city2", "city3"]
df2=spark.createDataFrame(data2, columns2)
display(df2)

city1,city2,city3
Goa,,AP
,AP,null
null,,Bglr


In [0]:
df22=(
    df2.withColumn('firstnotnullval',coalesce(when(df2.city1=="",None).otherwise(df2.city1)
                                              ,when(df2.city2=="",None).otherwise(df2.city2)
                                              ,when(df2.city3=="",None).otherwise(df2.city3))
                   )
)
df22.show()

+-----+-----+-----+---------------+
|city1|city2|city3|firstnotnullval|
+-----+-----+-----+---------------+
|  Goa|     |   AP|            Goa|
|     |   AP| null|             AP|
| null|     | Bglr|               |
+-----+-----+-----+---------------+



**3 percentage marks and result**

In [0]:

data1=[(1,"Steve"),(2,"David"),(3,"John"),(4,"Shree"),(5,"Helen")]
data2=[(1,"SQL",90),(1,"PySpark",100),(2,"SQL",70),(2,"PySpark",60),(3,"SQL",30),(3,"PySpark",20),(4,"SQL",50),(4,"PySpark",50),(5,"SQL",45),(5,"PySpark",45)]

schema1=["Id","Name"]
schema2=["Id","Subject","Mark"]

df1=spark.createDataFrame(data1,schema1)
df2=spark.createDataFrame(data2,schema2)
display(df1)
display(df2)

Id,Name
1,Steve
2,David
3,John
4,Shree
5,Helen


Id,Subject,Mark
1,SQL,90
1,PySpark,100
2,SQL,70
2,PySpark,60
3,SQL,30
3,PySpark,20
4,SQL,50
4,PySpark,50
5,SQL,45
5,PySpark,45


In [0]:
df_join=df1.join(df2,df1.Id==df2.Id,'inner').select(df1.Id,df1.Name,df2.Subject,df2.Mark)
df_join.show(4)

+---+-----+-------+----+
| Id| Name|Subject|Mark|
+---+-----+-------+----+
|  1|Steve|    SQL|  90|
|  1|Steve|PySpark| 100|
|  2|David|    SQL|  70|
|  2|David|PySpark|  60|
+---+-----+-------+----+
only showing top 4 rows



In [0]:
from pyspark.sql.window import Window
#windowspec=Window.partitionBy('Id','Name').orderBy('Mark')
df_perc=df_join.groupBy('Id','Name').agg((sum('Mark')/count('*')).alias('percentage'))
df_perc.show()

+---+-----+----------+
| Id| Name|percentage|
+---+-----+----------+
|  1|Steve|      95.0|
|  2|David|      65.0|
|  3| John|      25.0|
|  4|Shree|      50.0|
|  5|Helen|      45.0|
+---+-----+----------+



In [0]:
df_result=(
    df_perc.withColumn('result',when(col('percentage')>70,'Distinction')
                                .when((col('percentage')>60) & (col('percentage')<69),'First class')
                                .when((col('percentage')>50) & (col('percentage')<59),'Second class')
                                .when((col('percentage')>40) & (col('percentage')<49),'Third class')
                                .otherwise(lit('Fail'))
                       )
)

df_result.show()

+---+-----+----------+-----------+
| Id| Name|percentage|     result|
+---+-----+----------+-----------+
|  1|Steve|      95.0|Distinction|
|  2|David|      65.0|First class|
|  3| John|      25.0|       Fail|
|  4|Shree|      50.0|       Fail|
|  5|Helen|      45.0|Third class|
+---+-----+----------+-----------+



**4 employee manager self join**

In [0]:
#Employees Salary info
data1=[(100,"Raj",None,1,"01-04-23",50000),
       (200,"Joanne",100,1,"01-04-23",4000),(200,"Joanne",100,1,"13-04-23",4500),(200,"Joanne",100,1,"14-04-23",4020)]
schema1=["EmpId","EmpName","Mgrid","deptid","salarydt","salary"]
df_salary=spark.createDataFrame(data1,schema1)
display(df_salary)
#department dataframe
data2=[(1,"IT"),
       (2,"HR")]
schema2=["deptid","deptname"]
df_dept=spark.createDataFrame(data2,schema2)
display(df_dept)

EmpId,EmpName,Mgrid,deptid,salarydt,salary
100,Raj,null,1,01-04-23,50000
200,Joanne,100,1,01-04-23,4000
200,Joanne,100,1,13-04-23,4500
200,Joanne,100,1,14-04-23,4020


deptid,deptname
1,IT
2,HR


In [0]:
emp_join=df_salary.join(df_dept,df_salary.deptid==df_dept.deptid,'inner').drop(df_dept.deptid)
emp_join.show()

+-----+-------+-----+------+--------+------+--------+
|EmpId|EmpName|Mgrid|deptid|salarydt|salary|deptname|
+-----+-------+-----+------+--------+------+--------+
|  100|    Raj| null|     1|01-04-23| 50000|      IT|
|  200| Joanne|  100|     1|01-04-23|  4000|      IT|
|  200| Joanne|  100|     1|13-04-23|  4500|      IT|
|  200| Joanne|  100|     1|14-04-23|  4020|      IT|
+-----+-------+-----+------+--------+------+--------+



In [0]:
df_mngr=emp_join.alias('a').join(emp_join.alias('b'),col('a.Mgrid')==col('b.EmpId'),'left')\
    .select(   col('a.deptname'),
    col('b.EmpName').alias('ManagerName'),
    col('a.EmpName'),
    col('a.salarydt'),
    col('a.salary'))

df_mngr.show()    

+--------+-----------+-------+--------+------+
|deptname|ManagerName|EmpName|salarydt|salary|
+--------+-----------+-------+--------+------+
|      IT|       null|    Raj|01-04-23| 50000|
|      IT|        Raj| Joanne|01-04-23|  4000|
|      IT|        Raj| Joanne|13-04-23|  4500|
|      IT|        Raj| Joanne|14-04-23|  4020|
+--------+-----------+-------+--------+------+



In [0]:
from pyspark.sql.functions import to_date,month,year

df_mngr1=df_mngr.withColumn('newsalarydate',to_date(col('salarydt'),'dd-MM-yy')).drop('salarydt')
df_mngr2=(
    df_mngr1.withColumn('salary_month',month(col('newsalarydate')))
    .withColumn('salary_year',year(col('newsalarydate')))
    .drop('newsalarydate')
)
df_mngr2.show()

+--------+-----------+-------+------+------------+-----------+
|deptname|ManagerName|EmpName|salary|salary_month|salary_year|
+--------+-----------+-------+------+------------+-----------+
|      IT|       null|    Raj| 50000|           4|       2023|
|      IT|        Raj| Joanne|  4000|           4|       2023|
|      IT|        Raj| Joanne|  4500|           4|       2023|
|      IT|        Raj| Joanne|  4020|           4|       2023|
+--------+-----------+-------+------+------------+-----------+



**Nested json question**

In [0]:
json_str=""" 
{
 "Institute_Name" : "ABC_Coaching_Center",
 "Course_type" : "Best_seller" ,
 "branches" : [
  {
   "State" : "Maharashtra",
   "City" : "Mumbai",
   "address" : "XYZ"
  },
  {
   "State" : "Gujrat",
   "City" : "Surat",
   "address" : "PQRX"
  }
 ],
 "Head_Office_Contact" : 8787878787
}
"""

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import MapType,StructType,StructField,StringType,IntegerType,ArrayType,DecimalType

schema1=(
    StructType([
    StructField('name',StringType()),
    StructField('id',IntegerType()),
    StructField('address', StructType([
         StructField('city', StringType(), True),
         StructField('state', StringType(), True)
         ])),
     StructField('phone_numbers', ArrayType(
        StructType([
         StructField('number', StringType(), True),
         StructField('type', StringType(), True)
         ])
     ))
    ])
)

df_json=(
    spark.read.format('json')
    .option('multiLine',True)
    .option("header",True) 
    .option("schema",schema1)
    .load('dbfs:/FileStore/tables/nested_json.txt')

)
df_json.printSchema()
df_json.show(truncate=False)

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phone_numbers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- type: string (nullable = true)

+-------------------+---+----+--------------------------------------------+
|address            |id |name|phone_numbers                               |
+-------------------+---+----+--------------------------------------------+
|{New York, NY}     |1  |John|[{123-456-7890, home}, {987-654-3210, work}]|
|{San Francisco, CA}|2  |Jane|[{555-1234, mobile}, {777-4321, work}]      |
+-------------------+---+----+--------------------------------------------+



**Flight question**

In [0]:
flights_data = [(1,'Flight1' , 'Delhi' , 'Hyderabad'),
                (1,'Flight2' , 'Hyderabad' , 'Kochi'),
                (1,'Flight3' , 'Kochi' , 'Mangalore'),
                (2,'Flight1' , 'Mumbai' , 'Ayodhya'),
                (2,'Flight2' , 'Ayodhya' , 'Gorakhpur')
                ]

_schema = "cust_id int, flight_id string , origin string , destination string"

df_flight = spark.createDataFrame(data = flights_data , schema= _schema)
df_flight.show()

+-------+---------+---------+-----------+
|cust_id|flight_id|   origin|destination|
+-------+---------+---------+-----------+
|      1|  Flight1|    Delhi|  Hyderabad|
|      1|  Flight2|Hyderabad|      Kochi|
|      1|  Flight3|    Kochi|  Mangalore|
|      2|  Flight1|   Mumbai|    Ayodhya|
|      2|  Flight2|  Ayodhya|  Gorakhpur|
+-------+---------+---------+-----------+



In [0]:
from pyspark.sql.window import Window

windowspec=Window.partitionBy('cust_id').orderBy('flight_id')

df_flights=df_flight.withColumn('rnk',rank().over(windowspec))

In [0]:
display(df_flights)

cust_id,flight_id,origin,destination,rnk
1,Flight1,Delhi,Hyderabad,1
1,Flight2,Hyderabad,Kochi,2
1,Flight3,Kochi,Mangalore,3
2,Flight1,Mumbai,Ayodhya,1
2,Flight2,Ayodhya,Gorakhpur,2


In [0]:


df_flights1=df_flights.groupBy('cust_id').agg(min('rnk').alias('start'),
                                              max('rnk').alias('end'))

df_flights1.show()                                              


+-------+-----+---+
|cust_id|start|end|
+-------+-----+---+
|      1|    1|  3|
|      2|    1|  2|
+-------+-----+---+



In [0]:
df_flights.show(3)

+-------+---------+---------+-----------+---+
|cust_id|flight_id|   origin|destination|rnk|
+-------+---------+---------+-----------+---+
|      1|  Flight1|    Delhi|  Hyderabad|  1|
|      1|  Flight2|Hyderabad|      Kochi|  2|
|      1|  Flight3|    Kochi|  Mangalore|  3|
+-------+---------+---------+-----------+---+
only showing top 3 rows



In [0]:
df_flights2=(
    df_flights1.alias('a').join(df_flights.alias('b'),col('a.cust_id')==col('b.cust_id'),'inner')

).drop(col('b.cust_id'))
df_flights2.show()


+-------+-----+---+---------+---------+-----------+---+
|cust_id|start|end|flight_id|   origin|destination|rnk|
+-------+-----+---+---------+---------+-----------+---+
|      1|    1|  3|  Flight1|    Delhi|  Hyderabad|  1|
|      1|    1|  3|  Flight2|Hyderabad|      Kochi|  2|
|      1|    1|  3|  Flight3|    Kochi|  Mangalore|  3|
|      2|    1|  2|  Flight1|   Mumbai|    Ayodhya|  1|
|      2|    1|  2|  Flight2|  Ayodhya|  Gorakhpur|  2|
+-------+-----+---+---------+---------+-----------+---+



In [0]:
df_final=(df_flights2.groupBy('cust_id').agg(
    max(when(col('start')==col('rnk'),col('origin'))).alias('origin'),
     max(when(col('end')==col('rnk'),col('destination'))).alias('destination')
)
)
df_final.show()

+-------+------+-----------+
|cust_id|origin|destination|
+-------+------+-----------+
|      1| Delhi|  Mangalore|
|      2|Mumbai|  Gorakhpur|
+-------+------+-----------+



**Matching records**

In [0]:
source_data = [(1, 'A'), (2, 'B'), (3, 'C'), (4, 'D')]
source_columns = ['id','name']
target_data = [(1, 'A'), (2, 'B'), (4, 'X'), (5, 'F')]
target_columns = ['id','name']

source_df = spark.createDataFrame(data=source_data, schema=source_columns)
target_df = spark.createDataFrame(data=target_data, schema=target_columns)

In [0]:
source_df.show()
target_df.show()


+---+----+
| id|name|
+---+----+
|  1|   A|
|  2|   B|
|  3|   C|
|  4|   D|
+---+----+

+---+----+
| id|name|
+---+----+
|  1|   A|
|  2|   B|
|  4|   X|
|  5|   F|
+---+----+



In [0]:
df_full=(
    source_df.alias('s').join(target_df.alias('t'),col('s.id')==col('t.id'),'outer')
            .select(col('s.id').alias('source_id'),col('s.name').alias('source_name'),col('t.id').alias('target_id'),
                    col('t.name').alias('target_name'))
)
df_full.show()

+---------+-----------+---------+-----------+
|source_id|source_name|target_id|target_name|
+---------+-----------+---------+-----------+
|        1|          A|        1|          A|
|        2|          B|        2|          B|
|        3|          C|     null|       null|
|        4|          D|        4|          X|
|     null|       null|        5|          F|
+---------+-----------+---------+-----------+



In [0]:
#df_full=df_full.selectExpr('*').filter("source_name <> target_name")
df_full.show()

+---------+-----------+---------+-----------+
|source_id|source_name|target_id|target_name|
+---------+-----------+---------+-----------+
|        1|          A|        1|          A|
|        2|          B|        2|          B|
|        3|          C|     null|       null|
|        4|          D|        4|          X|
|     null|       null|        5|          F|
+---------+-----------+---------+-----------+



In [0]:
display(df_full.withColumn('comment',when((col('source_id')==col('target_id')) & (col('source_name')!=col('target_name')),lit('Mismatched'))
                                    .when(col('target_id').isNull(),lit('New in target'))
                                    .when(col('source_id').isNull(),lit('New in source'))
                           ))

source_id,source_name,target_id,target_name,comment
1,A,1,A,null
2,B,2,B,null
3,C,null,null,New in target
4,D,4,X,Mismatched
null,null,5,F,New in source


**Product by increasing sales each year**

In [0]:
sales_data = [
 (1, 2019, 1000.00),
 (1, 2020, 1200.00),
 (1, 2021, 1100.00),
 (2, 2019, 500.00),
 (2, 2020, 600.00),
 (2, 2021, 900.00),
 (3, 2019, 300.00),
 (3, 2020, 450.00),
 (3, 2021, 400.00)
 ]

sales_schema = ['product_id', 'year', 'total_sales_revenue']

sales_df = spark.createDataFrame(data = sales_data , schema = sales_schema)
sales_df.show()

+----------+----+-------------------+
|product_id|year|total_sales_revenue|
+----------+----+-------------------+
|         1|2019|             1000.0|
|         1|2020|             1200.0|
|         1|2021|             1100.0|
|         2|2019|              500.0|
|         2|2020|              600.0|
|         2|2021|              900.0|
|         3|2019|              300.0|
|         3|2020|              450.0|
|         3|2021|              400.0|
+----------+----+-------------------+



In [0]:
df_prod=(
    sales_df
    .withColumn('next_yr_sale',lead('total_sales_revenue',1,0).over(Window.partitionBy('product_id').orderBy('year')))
)

df_prod.show()

+----------+----+-------------------+------------+
|product_id|year|total_sales_revenue|next_yr_sale|
+----------+----+-------------------+------------+
|         1|2019|             1000.0|      1200.0|
|         1|2020|             1200.0|      1100.0|
|         1|2021|             1100.0|         0.0|
|         2|2019|              500.0|       600.0|
|         2|2020|              600.0|       900.0|
|         2|2021|              900.0|         0.0|
|         3|2019|              300.0|       450.0|
|         3|2020|              450.0|       400.0|
|         3|2021|              400.0|         0.0|
+----------+----+-------------------+------------+



In [0]:
df_prod1=df_prod.where('next_yr_sale > total_sales_revenue')
df_prod1.show()

+----------+----+-------------------+------------+
|product_id|year|total_sales_revenue|next_yr_sale|
+----------+----+-------------------+------------+
|         1|2019|             1000.0|      1200.0|
|         2|2019|              500.0|       600.0|
|         2|2020|              600.0|       900.0|
|         3|2019|              300.0|       450.0|
+----------+----+-------------------+------------+



**Swap name with even/odd**

In [0]:
_data = [(1,'Abbot'),(2,'Doris'),(3,'Emerson'),(4,'Green'),(5,'Jeames')]
_schema = ['id', 'student']


df = spark.createDataFrame(data = _data, schema=_schema)
df.show()


+---+-------+
| id|student|
+---+-------+
|  1|  Abbot|
|  2|  Doris|
|  3|Emerson|
|  4|  Green|
|  5| Jeames|
+---+-------+



In [0]:
#windowprev=Window.

df_mid=(
    df.withColumn('next_student',lead('student',1).over(Window.orderBy('id')))
          .withColumn('prev_student',lag('student',1).over(Window.orderBy('id')))
)

df_mid.show()

+---+-------+------------+------------+
| id|student|next_student|prev_student|
+---+-------+------------+------------+
|  1|  Abbot|       Doris|        null|
|  2|  Doris|     Emerson|       Abbot|
|  3|Emerson|       Green|       Doris|
|  4|  Green|      Jeames|     Emerson|
|  5| Jeames|        null|       Green|
+---+-------+------------+------------+



In [0]:
df_final=(
    df_mid.withColumn('new_student',when(col('id') % 2==0,col('prev_student'))
                                    .when((col('id') % 2!=0) & (col('next_student').isNull()),col('student'))
                                    .when(col('id') % 2!=0,col('next_student'))
                                                                     
                                    
                      )
)

df_final.select('id','new_student').show()

+---+-----------+
| id|new_student|
+---+-----------+
|  1|      Doris|
|  2|      Abbot|
|  3|      Green|
|  4|    Emerson|
|  5|     Jeames|
+---+-----------+



**Retrieve information of employees who has loggedin atleast 2 consecutive days**

In [0]:
_data = [(101,'02-01-2024','N'),
(101,'03-01-2024','Y'),
(101,'04-01-2024','N'),
(101,'07-01-2024','Y'),
(102,'01-01-2024','N'),
(102,'02-01-2024','Y'),
(102,'03-01-2024','Y'),
(102,'04-01-2024','N'),
(102,'05-01-2024','Y'),
(102,'06-01-2024','Y'),
(102,'07-01-2024','Y'),
(103,'01-01-2024','N'),
(103,'04-01-2024','N'),
(103,'05-01-2024','Y'),
(103,'06-01-2024','Y'),
(103,'07-01-2024','N')
]
_schema = ["emp_id" , "log_date" , "flag"]

df = spark.createDataFrame(data = _data , schema = _schema)
df.show()


+------+----------+----+
|emp_id|  log_date|flag|
+------+----------+----+
|   101|02-01-2024|   N|
|   101|03-01-2024|   Y|
|   101|04-01-2024|   N|
|   101|07-01-2024|   Y|
|   102|01-01-2024|   N|
|   102|02-01-2024|   Y|
|   102|03-01-2024|   Y|
|   102|04-01-2024|   N|
|   102|05-01-2024|   Y|
|   102|06-01-2024|   Y|
|   102|07-01-2024|   Y|
|   103|01-01-2024|   N|
|   103|04-01-2024|   N|
|   103|05-01-2024|   Y|
|   103|06-01-2024|   Y|
|   103|07-01-2024|   N|
+------+----------+----+



In [0]:
df.printSchema()

root
 |-- emp_id: long (nullable = true)
 |-- log_date: string (nullable = true)
 |-- flag: string (nullable = true)



In [0]:
df_filtered=df.filter("flag='Y'")
df_filtered.show()

+------+----------+----+
|emp_id|  log_date|flag|
+------+----------+----+
|   101|03-01-2024|   Y|
|   101|07-01-2024|   Y|
|   102|02-01-2024|   Y|
|   102|03-01-2024|   Y|
|   102|05-01-2024|   Y|
|   102|06-01-2024|   Y|
|   102|07-01-2024|   Y|
|   103|05-01-2024|   Y|
|   103|06-01-2024|   Y|
+------+----------+----+



In [0]:
df_filtered_rn=(
    df_filtered.withColumn('rnk',rank().over(Window.partitionBy('emp_id').orderBy('log_date')))
)

df_filtered_rn.show()

+------+----------+----+---+
|emp_id|  log_date|flag|rnk|
+------+----------+----+---+
|   101|03-01-2024|   Y|  1|
|   101|07-01-2024|   Y|  2|
|   102|02-01-2024|   Y|  1|
|   102|03-01-2024|   Y|  2|
|   102|05-01-2024|   Y|  3|
|   102|06-01-2024|   Y|  4|
|   102|07-01-2024|   Y|  5|
|   103|05-01-2024|   Y|  1|
|   103|06-01-2024|   Y|  2|
+------+----------+----+---+



In [0]:
from pyspark.sql.functions import to_date,dayofmonth

_df_day=df_filtered_rn.withColumn('day',dayofmonth(to_date('log_date','dd-mm-yyyy')))

_df_day.show()

+------+----------+----+---+---+
|emp_id|  log_date|flag|rnk|day|
+------+----------+----+---+---+
|   101|03-01-2024|   Y|  1|  3|
|   101|07-01-2024|   Y|  2|  7|
|   102|02-01-2024|   Y|  1|  2|
|   102|03-01-2024|   Y|  2|  3|
|   102|05-01-2024|   Y|  3|  5|
|   102|06-01-2024|   Y|  4|  6|
|   102|07-01-2024|   Y|  5|  7|
|   103|05-01-2024|   Y|  1|  5|
|   103|06-01-2024|   Y|  2|  6|
+------+----------+----+---+---+



In [0]:
_df_day1=_df_day.withColumn('group',col('day')-col('rnk'))
_df_day1.show()

+------+----------+----+---+---+-----+
|emp_id|  log_date|flag|rnk|day|group|
+------+----------+----+---+---+-----+
|   101|03-01-2024|   Y|  1|  3|    2|
|   101|07-01-2024|   Y|  2|  7|    5|
|   102|02-01-2024|   Y|  1|  2|    1|
|   102|03-01-2024|   Y|  2|  3|    1|
|   102|05-01-2024|   Y|  3|  5|    2|
|   102|06-01-2024|   Y|  4|  6|    2|
|   102|07-01-2024|   Y|  5|  7|    2|
|   103|05-01-2024|   Y|  1|  5|    4|
|   103|06-01-2024|   Y|  2|  6|    4|
+------+----------+----+---+---+-----+



In [0]:
_df_day2=_df_day1.groupBy('emp_id','group').agg(count('*').alias('count')).filter("count>=2")

_df_day2.show()

+------+-----+-----+
|emp_id|group|count|
+------+-----+-----+
|   102|    1|    2|
|   102|    2|    3|
|   103|    4|    2|
+------+-----+-----+

